In [1]:
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

# plt.style.use('ggplot')

# Import

First we open a file containing a dictionary of columns and associated data types generated in `../../optimize/`, then we read input file.

In [2]:
with open('../input/dtypes.yaml', 'r') as yamlfile:
    column_types = yaml.load(yamlfile)

read_csv_opts = {'sep': '|',
                 'quotechar': '"',
                 'compression': 'gzip',
                 'encoding': 'utf-8',
                 'dtype': column_types,
                 'parse_dates': ['MissionDate'],
                 'infer_datetime_format': True}

df = pd.read_csv('../input/ice-air.csv.gz', **read_csv_opts)

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732625 entries, 0 to 1732624
Data columns (total 48 columns):
Status                          category
Sex                             category
Convictions                     category
GangMember                      category
ClassLvl                        float32
Age                             float32
MissionDate                     datetime64[ns]
MissionNumber                   uint32
PULOC                           category
DropLoc                         category
StrikeFromList                  float32
ReasonStruck                    category
R-T                             category
Code                            category
CountryOfCitizenship            category
Juvenile                        bool
MissionWeek                     uint8
MissionQuarter                  uint8
MissionYear                     uint16
MissionMonth                    uint8
Criminality                     category
FamilyUnitFlag                  float32


In [3]:
with open('../input/airport_dict.yaml', 'r') as yamlfile:
    airport_dict = yaml.load(yamlfile)

# Status

In [4]:
removals = df['R-T'] == 'R'
transfers = df['R-T'] == 'T'

In [5]:
status_count = df[removals].drop_duplicates(subset='AlienMasterID')['Status'].str.upper().value_counts(dropna=False)

In [6]:
status = pd.read_csv('../../share/resources/status.csv')

In [7]:
valid_status_codes = list(status['Code'])

In [8]:
status_dict = dict(zip(status['Code'], status['Status']))

In [9]:
pending = ['2A', '2B', '8A', '8B', '8D']
benefit = ['5C', '5D']
expedited_admin_rein = ['8F', '8G', '8H', '8I', '11', '16']
problematic = pending + benefit + expedited_admin_rein

In [10]:
problematic_count = 0
valid = df[removals]['Status'].str.upper().isin(valid_status_codes)
valid_count = sum(valid)
invalid = ~df[removals]['Status'].str.upper().isin(valid_status_codes)
invalid_count = sum(invalid)

print(f'ICE Air - All removals')
print()
print(f'ICE status codes for passengers on direct removal flights.')
print(f'(Statuses which may raise due process concerns starred)')
print()
print(f'{valid_count} valid values.')
print(f'{invalid_count} invalid values.')
print()
for c in valid_status_codes:
    
    try:
        count = status_count[c]
    except KeyError:
        count = 0
    
    if c in problematic:
        problematic_count = problematic_count + count
        print(f'[{c}]: *{count}* ({status_dict[c]})')
    else:
        print(f'[{c}]: {count} ({status_dict[c]})')
        
assert sum(df[removals]['Status'].isin(problematic)) == problematic_count
print()
print(f'{problematic_count} total problematic cases.')
print(f"{sum(df.loc[removals, 'Status'].isin(pending))} with pending appeals.")
print(f"{sum(df.loc[removals, 'Status'].isin(benefit))} with benefit blocking deportation.")
print(f"{sum(df.loc[removals, 'Status'].isin(expedited_admin_rein))} under expedited, administrative, reinstatement of removal.")

ICE Air - All removals

ICE status codes for passengers on direct removal flights.
(Statuses which may raise due process concerns starred)

839344 valid values.
394568 invalid values.

[1A]: 55104 (Voluntary Departure – Un-Expired and Un-Extended Voluntary Departure.)
[1B]: 284 (Voluntary Departure - Extended Departure Period)
[1C]: 8 (Exipred Voluntary Departure Period - Referred to Investigation)
[2A]: *2223* (Deportable – Under Adjudication by IJ)
[2B]: *96* (Deportable – Under Adjudication by BIA)
[3]: 20572 (Deportable – Administratively Final Order)
[5A]: 14 (Referred for Investigation - No Show for Hearing - No Final O...)
[5B]: 797 (Removable – ICE Fugitive)
[5C]: *16* (Relief Granted – Withholding of Deportation/Removal)
[5D]: *86* (Final Order of Deportation/Removal – Deferred Action Granted.)
[5E]: 32 (Relief Granted – Extended Voluntary Departure)
[5F]: 57 (Unable to Obtain Travel Document)
[8A]: *1489* (Excludable/Inadmissible – Hearing Not Commenced)
[8B]: *4080* (Excluda

In [11]:
sum(df[removals]['Status'].isin(expedited_admin_rein))

627694

In [12]:
invalid_status = df[~df['Status'].isin(valid_status_codes)]
invalid_status_counts = invalid_status['Status'].value_counts()
# bad_statuses = list(status_counts[status_counts > 0].index)

In [13]:
invalid_status_counts.head(25)

REMOVAL           231309
RM                164378
NO STATUS          50831
NTA                20931
TRANSFER - F/O     19507
TRANSFER - NTA      9795
REINSTATE           6819
ER-TRANSFER         5917
STAGER              4771
NTA - TRANSFER      3036
F/O - TRANSFER      2620
TRANSFER            2174
REINST              1800
TRANSFER-NTA        1468
WD                  1030
FINAL ORDER          909
JUVI                 845
1/15/1900            747
NTA-TRANSFER         680
WA/NTA               666
NTA-TRFD             663
V/R                  635
DEPORT               584
REIN                 569
RE-INSTATE           484
Name: Status, dtype: int64

## Status by FY for removals & transfers

In [14]:
status_fy = df[removals].drop_duplicates(subset='AlienMasterID').resample('AS-OCT', on='MissionDate')['Status'].value_counts().unstack()

In [15]:
total = df[removals].drop_duplicates(subset='AlienMasterID').resample('AS-OCT', on='MissionDate')['AlienMasterID'].nunique()

In [16]:
status_fy[expedited_admin_rein]

Status,8F,8G,8H,8I,11,16
MissionDate,,,,,,
2010-10-01,9899.0,1.0,NaN,NaN,174.0,6484.0
2011-10-01,22038.0,NaN,NaN,NaN,92.0,13747.0
2012-10-01,52132.0,462.0,9.0,5.0,2298.0,45536.0
2013-10-01,53894.0,2556.0,164.0,33.0,3008.0,50542.0
2014-10-01,32056.0,4049.0,61.0,80.0,3832.0,40019.0
2015-10-01,29940.0,10335.0,246.0,128.0,3741.0,42121.0
2016-10-01,26146.0,13062.0,1264.0,214.0,2913.0,41956.0
2017-10-01,32839.0,12492.0,349.0,276.0,2652.0,45714.0
2018-10-01,6290.0,2686.0,38.0,45.0,399.0,8677.0


In [17]:
# status_fy[valid_status_codes].to_csv('../output/valid_status_by_year.csv')

In [18]:
all_others = status_fy[status_fy.columns[~status_fy.columns.isin(valid_status_codes)]].sum(axis=1)
all_others.name = 'all_others'

In [19]:
status_table = pd.concat([status_fy[valid_status_codes], all_others], axis=1).T

In [20]:
status_table

MissionDate,2010-10-01 00:00:00,2011-10-01 00:00:00,2012-10-01 00:00:00,2013-10-01 00:00:00,2014-10-01 00:00:00,2015-10-01 00:00:00,2016-10-01 00:00:00,2017-10-01 00:00:00,2018-10-01 00:00:00
1A,22425.0,22071.0,9738.0,513.0,264.0,36.0,23.0,27.0,7.0
1B,NaN,NaN,112.0,93.0,72.0,1.0,1.0,4.0,1.0
1C,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN,NaN
2A,NaN,NaN,994.0,233.0,301.0,106.0,217.0,325.0,47.0
2B,NaN,NaN,15.0,13.0,11.0,15.0,24.0,15.0,3.0
3,1149.0,555.0,1281.0,1737.0,2645.0,3354.0,4033.0,4953.0,865.0
5A,NaN,NaN,2.0,4.0,3.0,NaN,2.0,2.0,1.0
5B,NaN,NaN,68.0,241.0,141.0,65.0,133.0,137.0,12.0
5C,NaN,NaN,1.0,4.0,4.0,1.0,2.0,3.0,1.0
5D,NaN,NaN,16.0,18.0,8.0,4.0,19.0,17.0,4.0


In [21]:
# status_table.to_csv('../output/removal_status_by_year.csv')
# (status_table/total).to_csv('../output/removal_status_by_year_percent.csv')

## Status for specific airport

In [90]:
airport = 'KBFI'

In [91]:
airport_name = airport_dict[airport]
pickups = df['PULOC'] == airport
dropoffs = df['DropLoc'] == airport
airport_df = df[pickups|dropoffs]

In [92]:
removals = airport_df['R-T'] == 'R'
transfers = airport_df['R-T'] == 'T'

In [93]:
status_count = airport_df[removals].drop_duplicates(subset='AlienMasterID')['Status'].str.upper().value_counts(dropna=False)
count = 0
problematic_count = 0
valid = airport_df[removals]['Status'].str.upper().isin(valid_status_codes)
valid_count = sum(valid)
invalid = ~airport_df[removals]['Status'].str.upper().isin(valid_status_codes)
invalid_count = sum(invalid)

print(f'ICE Air - {airport} removals')
print()
print(f'ICE status codes for passengers on direct removal flights.')
print(f'(Statuses which may raise due process concerns starred)')
print()
print(f'{valid_count} valid values.')
print(f'{invalid_count} invalid values.')
print()
for c in valid_status_codes:
    
    try:
        count = status_count[c]
    except KeyError:
        count = 0
    
    if c in problematic:
        problematic_count = problematic_count + count
        print(f'[{c}]: *{count}* ({status_dict[c]})')
    else:
        print(f'[{c}]: {count} ({status_dict[c]})')
        
assert sum(airport_df[removals]['Status'].isin(problematic)) == problematic_count
print()
print(f'{problematic_count} total problematic cases.')
print(f"{sum(airport_df.loc[removals, 'Status'].isin(pending))} with pending appeals.")
print(f"{sum(airport_df.loc[removals, 'Status'].isin(benefit))} with benefit blocking deportation.")
print(f"{sum(airport_df.loc[removals, 'Status'].isin(expedited_admin_rein))} under expedited, administrative, reinstatement of removal.")

ICE Air - KBFI removals

ICE status codes for passengers on direct removal flights.
(Statuses which may raise due process concerns starred)

7528 valid values.
8541 invalid values.

[1A]: 2446 (Voluntary Departure – Un-Expired and Un-Extended Voluntary Departure.)
[1B]: 20 (Voluntary Departure - Extended Departure Period)
[1C]: 0 (Exipred Voluntary Departure Period - Referred to Investigation)
[2A]: *32* (Deportable – Under Adjudication by IJ)
[2B]: *1* (Deportable – Under Adjudication by BIA)
[3]: 161 (Deportable – Administratively Final Order)
[5A]: 1 (Referred for Investigation - No Show for Hearing - No Final O...)
[5B]: 20 (Removable – ICE Fugitive)
[5C]: *1* (Relief Granted – Withholding of Deportation/Removal)
[5D]: *1* (Final Order of Deportation/Removal – Deferred Action Granted.)
[5E]: 1 (Relief Granted – Extended Voluntary Departure)
[5F]: 0 (Unable to Obtain Travel Document)
[8A]: *31* (Excludable/Inadmissible – Hearing Not Commenced)
[8B]: *86* (Excludable/Inadmissible – U

In [99]:
airport_df[removals][valid]['Status'].value_counts().head(23)

1A    2446
16    1781
8C    1077
12     786
11     289
8G     272
8F     243
9      214
3      161
8B      86
2A      32
8A      31
8I      28
8E      28
1B      20
5B      20
8D       7
8H       2
5A       1
5D       1
5E       1
5C       1
2B       1
Name: Status, dtype: int64

# Status for specific nationality

In [24]:
nationality = 'CAMBODIA'

errormsg = f'{nationality} not represented in dataset'
assert nationality.upper() in df['CountryOfCitizenship'].str.upper().unique(), errormsg

nationality_df = df[df['CountryOfCitizenship'] == nationality.upper()]

removals = nationality_df['R-T'] == 'R'
transfers = nationality_df['R-T'] == 'T'

In [25]:
status_count = nationality_df[removals].drop_duplicates(subset='AlienMasterID')['Status'].str.upper().value_counts(dropna=False)
count = 0
problematic_count = 0
valid = nationality_df[removals]['Status'].str.upper().isin(valid_status_codes)
valid_count = sum(valid)
invalid = ~nationality_df[removals]['Status'].str.upper().isin(valid_status_codes)
invalid_count = sum(invalid)

print(f'ICE Air - {nationality} removals')
print()
print(f'ICE status codes for passengers on direct removal flights.')
print(f'(Statuses which may raise due process concerns starred)')
print()
print(f'{valid_count} valid values.')
print(f'{invalid_count} invalid values.')
print()
for c in valid_status_codes:
    
    try:
        count = status_count[c]
    except KeyError:
        count = 0
    
    if c in problematic:
        problematic_count = problematic_count + count
        print(f'[{c}]: *{count}* ({status_dict[c]})')
    else:
        print(f'[{c}]: {count} ({status_dict[c]})')
        
assert sum(nationality_df[removals]['Status'].isin(problematic)) == problematic_count
print()
print(f'{problematic_count} total problematic cases.')
print(f"{sum(nationality_df.loc[removals, 'Status'].isin(pending))} with pending appeals.")
print(f"{sum(nationality_df.loc[removals, 'Status'].isin(benefit))} with benefit blocking deportation.")
print(f"{sum(nationality_df.loc[removals, 'Status'].isin(expedited_admin_rein))} under expedited, administrative, reinstatement of removal.")

ICE Air - CAMBODIA removals

ICE status codes for passengers on direct removal flights.
(Statuses which may raise due process concerns starred)

120 valid values.
0 invalid values.

[1A]: 0 (Voluntary Departure – Un-Expired and Un-Extended Voluntary Departure.)
[1B]: 0 (Voluntary Departure - Extended Departure Period)
[1C]: 0 (Exipred Voluntary Departure Period - Referred to Investigation)
[2A]: *0* (Deportable – Under Adjudication by IJ)
[2B]: *0* (Deportable – Under Adjudication by BIA)
[3]: 105 (Deportable – Administratively Final Order)
[5A]: 0 (Referred for Investigation - No Show for Hearing - No Final O...)
[5B]: 2 (Removable – ICE Fugitive)
[5C]: *0* (Relief Granted – Withholding of Deportation/Removal)
[5D]: *0* (Final Order of Deportation/Removal – Deferred Action Granted.)
[5E]: 0 (Relief Granted – Extended Voluntary Departure)
[5F]: 5 (Unable to Obtain Travel Document)
[8A]: *0* (Excludable/Inadmissible – Hearing Not Commenced)
[8B]: *0* (Excludable/Inadmissible – Under Adj

In [26]:
pending = ['2A', '2B', '8A', '8B', '8D']
benefit = ['5C', '5D']
expedited_admin_rein = ['8F', '8G', '8H', '8I', '11', '16']

In [27]:
sum(nationality_df.loc[removals, 'Status'].isin(pending))

0

In [28]:
sum(nationality_df.loc[removals, 'Status'].isin(benefit))

0

In [29]:
sum(nationality_df.loc[removals, 'Status'].isin(expedited_admin_rein))

1

In [30]:
invalid_status = nationality_df[~nationality_df['Status'].isin(valid_status_codes)]
invalid_status_counts = invalid_status['Status'].value_counts()
# bad_statuses = list(status_counts[status_counts > 0].index)

In [31]:
invalid_status_counts.head(10)

TRANSFER - F/O    143
TRANSFER           67
TRANSFER - NTA     40
NO STATUS          39
REMOVAL            33
NTA                10
F/O - TRANSFER      7
STAGING             4
INTERVIEW           2
RR - TRANSFER       2
Name: Status, dtype: int64

# Status by AOR

In [32]:
df.columns

Index(['Status', 'Sex', 'Convictions', 'GangMember', 'ClassLvl', 'Age',
       'MissionDate', 'MissionNumber', 'PULOC', 'DropLoc', 'StrikeFromList',
       'ReasonStruck', 'R-T', 'Code', 'CountryOfCitizenship', 'Juvenile',
       'MissionWeek', 'MissionQuarter', 'MissionYear', 'MissionMonth',
       'Criminality', 'FamilyUnitFlag', 'UnaccompaniedFlag', 'AlienMasterID',
       'MissionID', 'air_AirportID', 'air_AirportName', 'air_City',
       'st_StateID', 'st_StateAbbr', 'AOR_AORID', 'AOR_AOR', 'AOR_AORName',
       'air_Country', 'air2_AirportID', 'air2_AirportName', 'air2_City',
       'st2_StateID', 'st2_StateAbbr', 'aor2_AORID', 'aor2_AOR',
       'aor2_AORName', 'air2_Country', 'air_LatitudeDecimalDegrees',
       'air_LongitudeDecimalDegrees', 'air2_LatitudeDecimalDegrees',
       'air2_LongitudeDecimalDegrees', 'NonCriminal'],
      dtype='object')

In [48]:
removals = df['R-T'] == 'R'
transfers = df['R-T'] == 'T'

In [101]:
df[removals].groupby(['AOR_AORID'])['Status'].value_counts()

/Users/philneff/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [102]:
aor = 22

In [103]:
aor_df = df[df['AOR_AORID'] == aor]

In [104]:
aor_df['Status'].value_counts().head(10)

REMOVAL           8206
16                6846
8C                4762
1A                2810
8F                2398
TRANSFER - F/O    2031
8G                1730
12                1363
11                1278
RM                 744
Name: Status, dtype: int64

In [105]:
status_count = aor_df.drop_duplicates(subset='AlienMasterID')['Status'].str.upper().value_counts(dropna=False)
count = 0
problematic_count = 0
valid = aor_df['Status'].str.upper().isin(valid_status_codes)
valid_count = sum(valid)
invalid = ~aor_df['Status'].str.upper().isin(valid_status_codes)
invalid_count = sum(invalid)

print(f'ICE Air - AOR {aor} removals')
print()
print(f'ICE status codes for passengers on direct removal flights.')
print(f'(Statuses which may raise due process concerns starred)')
print()
print(f'{valid_count} valid values.')
print(f'{invalid_count} invalid values.')
print()
for c in valid_status_codes:
    
    try:
        count = status_count[c]
    except KeyError:
        count = 0
    
    if c in problematic:
        problematic_count = problematic_count + count
        print(f'[{c}]: *{count}* ({status_dict[c]})')
    else:
        print(f'[{c}]: {count} ({status_dict[c]})')
        
assert sum(aor_df.loc[:, 'Status'].isin(problematic)) == problematic_count
print()
print(f'{problematic_count} total problematic cases.')
print(f"{sum(aor_df.loc[:, 'Status'].isin(pending))} with pending appeals.")
print(f"{sum(aor_df.loc[:, 'Status'].isin(benefit))} with benefit blocking deportation.")
print(f"{sum(aor_df.loc[:, 'Status'].isin(expedited_admin_rein))} under expedited, administrative, reinstatement of removal.") 

ICE Air - AOR 22 removals

ICE status codes for passengers on direct removal flights.
(Statuses which may raise due process concerns starred)

23445 valid values.
11317 invalid values.

[1A]: 2810 (Voluntary Departure – Un-Expired and Un-Extended Voluntary Departure.)
[1B]: 100 (Voluntary Departure - Extended Departure Period)
[1C]: 0 (Exipred Voluntary Departure Period - Referred to Investigation)
[2A]: *335* (Deportable – Under Adjudication by IJ)
[2B]: *12* (Deportable – Under Adjudication by BIA)
[3]: 633 (Deportable – Administratively Final Order)
[5A]: 29 (Referred for Investigation - No Show for Hearing - No Final O...)
[5B]: 93 (Removable – ICE Fugitive)
[5C]: *3* (Relief Granted – Withholding of Deportation/Removal)
[5D]: *2* (Final Order of Deportation/Removal – Deferred Action Granted.)
[5E]: 2 (Relief Granted – Extended Voluntary Departure)
[5F]: 13 (Unable to Obtain Travel Document)
[8A]: *102* (Excludable/Inadmissible – Hearing Not Commenced)
[8B]: *283* (Excludable/Inadm

In [111]:
aor_df[valid]['Status'].value_counts().head(28)

16    6846
8C    4762
1A    2810
8F    2398
8G    1730
12    1363
11    1278
3      633
9      351
2A     335
8B     283
8E     175
8A     102
1B     100
5B      93
8I      70
5A      29
8D      27
8H      19
5F      13
2B      12
10       6
5C       3
5D       2
5E       2
13       1
14       1
15       1
Name: Status, dtype: int64

In [124]:
arts_count = aor_df[valid]['Status'].value_counts().head(28)

# ICE Seattle AOR removals

In [68]:
seattle_er = pd.read_csv('../frozen/SeattleAORExpeditedRemovals.csv', header=6)
seattle_or = pd.read_csv('../frozen/SeattleAOROtherRemovals.csv', header=6)

In [71]:
seattle_er.head()

,Departed Date,Date of Birth,Gender,Country of Citizenship,Event Number,Most Serious Conviction,Case Category
0,1/3/2012,"(b)(6),(b)(7)(c )",Male,FRANCE,(b)(7)(E ),NaN,[8F] Expedited Removal
1,1/5/2012,"(b)(6),(b)(7)(c )",Female,EL SALVADOR,(b)(7)(E ),NaN,[8G] Expedited Removal - Credible Fear Referral
2,1/6/2012,"(b)(6),(b)(7)(c )",Female,BELIZE,(b)(7)(E ),NaN,[8F] Expedited Removal
3,1/11/2012,"(b)(6),(b)(7)(c )",Female,EL SALVADOR,(b)(7)(E ),NaN,[8G] Expedited Removal - Credible Fear Referral
4,1/11/2012,"(b)(6),(b)(7)(c )",Female,EL SALVADOR,(b)(7)(E ),NaN,[8F] Expedited Removal


In [72]:
seattle_or.head()

,Departed Date,Date of Birth,Gender,Country of Citizenship,Event Number,Most Serious Conviction,Case Category
0,1/3/2012,"(b)(6),(b)(7)(c )",Male,MEXICO,(b)(7)(E ),NaN,[8C] Excludable / Inadmissible - Administrativ...
1,1/3/2012,"(b)(6),(b)(7)(c )",Female,MEXICO,(b)(7)(E ),NaN,[3] Deportable - Administratively Final Order
2,1/4/2012,"(b)(6),(b)(7)(c )",Male,MEXICO,(b)(7)(E ),NaN,[8C] Excludable / Inadmissible - Administrativ...
3,1/4/2012,"(b)(6),(b)(7)(c )",Male,MEXICO,(b)(7)(E ),NaN,[16] Reinstated Final Order
4,1/4/2012,"(b)(6),(b)(7)(c )",Male,MEXICO,(b)(7)(E ),NaN,[16] Reinstated Final Order


In [76]:
seattle_aor = pd.concat([seattle_er, seattle_or])

In [80]:
seattle_aor['Case Category'].value_counts()

[8C] Excludable / Inadmissible - Administrative Final Order Issued        9535
[16] Reinstated Final Order                                               6866
[8F] Expedited Removal                                                    3602
[8G] Expedited Removal - Credible Fear Referral                           2865
[3] Deportable - Administratively Final Order                             1871
[11] Administrative Deportation / Removal                                  932
[9] VR Under Safeguards                                                    539
[8E] Inadmissible - ICE Fugitive                                           206
[10] Visa Waiver Deportation / Removal                                     140
[8A] Excludable / Inadmissible - Hearing Not Commenced                      86
[2A] Deportable - Under Adjudication by IJ                                  64
[5B] Removable - ICE Fugitive                                               53
[14] Crewmen, Stowaways, S-Visa Holders, 235(c) Case

In [81]:
seattle_aor['Departed Date'] = pd.to_datetime(seattle_aor['Departed Date'])

In [84]:
seattle_aor['Departed Date'].describe()

count                   26913
unique                   2195
top       2013-01-09 00:00:00
freq                      150
first     2011-11-30 00:00:00
last      2019-07-02 00:00:00
Name: Departed Date, dtype: object

In [117]:
temp = pd.read_csv('../frozen/SeattleAORtemp.csv')

In [125]:
ero_count = temp.set_index('Status')

In [141]:
ero_count = ero_count.rename({'Count': 'ERO-LESA count'}, axis=1)

In [134]:
arts_count = arts_count.reset_index().set_index('index')

In [147]:
arts_count = arts_count.rename({'Status': 'ARTS count'}, axis=1)

In [152]:
compare = arts_count.join(ero_count)

In [153]:
compare.index.name = 'Status'

In [156]:
compare = compare.fillna(0)

In [158]:
compare['ERO-LESA count'] = compare['ERO-LESA count'].astype(int)

In [159]:
print(compare)

        ARTS-count  ERO-LESA count
Status                            
16            6846            6866
8C            4762            9535
1A            2810              13
8F            2398            3602
8G            1730            2865
12            1363               3
11            1278             932
3              633            1871
9              351             539
2A             335              64
8B             283              10
8E             175             206
8A             102              86
1B             100               1
5B              93              53
8I              70              40
5A              29               0
8D              27               0
8H              19              22
5F              13               2
2B              12               4
10               6             140
5C               3               3
5D               2               2
5E               2              11
13               1               0
14               1  